In [1]:
import pandas as pd
import numpy as np
import datetime

# 절대 모멘텀 
- 파일 로드
- 파생 변수 'STD-YM' 생성 -> Date 컬럼의 년-월 추출 대입
- 년, 월 별 마지막날 month_last_dt 데이터프레임 삽입
- 전월의 종가의 값을 가지는 파생변수 생성
- 전년도의 종가의 값을 가지는 파생변수 생성
- 전월, 전년도의 종가를 가지고 거래내역 생성
- 수익률 계산

In [2]:
df = pd.read_csv('../csv/SPY.csv')
df.head(1)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1993-01-29,43.96875,43.96875,43.75,43.9375,26.706757,1003200


In [3]:
# 새로운 파생변수 STD-YM 생성
# Date 컬럼 기준으로 년-월 추출해서 파생변수 대입 (pd.to_datetime())
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['STD-YM'] = df['Date'].dt.strftime('%Y-%m')

In [4]:
month_last_df = pd.DataFrame()

month_list = df['STD-YM'].unique()

for i in month_list:
    last_df = df[df['STD-YM'] == i].tail(1)
    month_last_df = pd.concat([month_last_df, last_df])
month_last_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 0 to 6647
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       318 non-null    datetime64[ns]
 1   Open       318 non-null    float64       
 2   High       318 non-null    float64       
 3   Low        318 non-null    float64       
 4   Close      318 non-null    float64       
 5   Adj Close  318 non-null    float64       
 6   Volume     318 non-null    int64         
 7   STD-YM     318 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 22.4+ KB


In [5]:
# 년-월 별 마지막날의 데이터들을 추출
# 새로운 데이터프레임에 결합

month_last_df = df[~(df.shift(-1)['STD-YM'] == df['STD-YM'])]
month_last_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 318 entries, 0 to 6647
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       318 non-null    datetime64[ns]
 1   Open       318 non-null    float64       
 2   High       318 non-null    float64       
 3   Low        318 non-null    float64       
 4   Close      318 non-null    float64       
 5   Adj Close  318 non-null    float64       
 6   Volume     318 non-null    int64         
 7   STD-YM     318 non-null    object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 22.4+ KB


In [6]:
# 'Date' 컬럼을 인덱스 변환

month_last_df.set_index(['Date'], inplace=True)

In [7]:
# month_last_df 파생변수 2개 생성
# 'BF_1M' 전월의 종가(Adj Close)의 값
# 'BF_12M' 전년도의 종가(Adj Close)의 값
# NaN이 나온다면 0으로 대체

month_last_df.head(2)

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1993-01-29,43.96875,43.96875,43.7500,43.93750,26.706757,1003200,1993-01
1993-02-26,44.43750,44.43750,44.1875,44.40625,26.991682,66200,1993-02


In [8]:
month_last_df['BF_1M'] = month_last_df.shift(1)['Adj Close'].fillna(0)
month_last_df['BF_12M'] = month_last_df.shift(12)['Adj Close'].fillna(0)

C:\Users\seopa\AppData\Local\Temp\ipykernel_9776\2005057084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_last_df['BF_1M'] = month_last_df.shift(1)['Adj Close'].fillna(0)
C:\Users\seopa\AppData\Local\Temp\ipykernel_9776\2005057084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_last_df['BF_12M'] = month_last_df.shift(12)['Adj Close'].fillna(0)


In [9]:
month_last_df.iloc[12:18]

,Open,High,Low,Close,Adj Close,Volume,STD-YM,BF_1M,BF_12M
Date,,,,,,,,,
1994-01-31,48.062500,48.31250,48.00000,48.21875,30.045223,313800,1994-01,29.032679,26.706757
1994-02-28,46.937500,47.06250,46.81250,46.81250,29.168985,333000,1994-02,30.045223,26.991682
1994-03-31,44.468750,44.68750,43.53125,44.59375,27.946764,788800,1994-03,29.168985,27.596252
1994-04-29,44.875000,45.15625,44.81250,45.09375,28.260098,481900,1994-04,27.946764,26.890142
1994-05-31,45.734375,45.90625,45.65625,45.81250,28.710556,160000,1994-05,28.260098,27.615356
1994-06-30,44.828125,44.84375,44.31250,44.46875,28.052668,271900,1994-06,28.710556,27.714964


# loc, iloc
1. loc
    - index와 column의 값을 비교하여 True인 행과 열을 출력
    - loc[index, column]
    - loc[1] -> index의 '값'이 1인 행 출력
    - loc[:,column명]
2. iloc
    - index와 column의 위치를 출력
    - 데이터프레임의 2번째 행 출력 -> iloc[1]
    - 데이터프레임에서 2번째 열 출력 -> iloc[:,1]

In [10]:
df.set_index(['Date'], inplace=True)

In [11]:
# 거래내역 추가
# 구매 조건 -> 전월 종가 / 전년도 종가 - 1의 값이 0보다 크고 무한대가 아닌 경우

df['trade'] = ''

for i in month_last_df.index:
    signal = ''

    # 절대 모멘텀 계산
    momentum_index = month_last_df.loc[i, 'BF_1M'] / month_last_df.loc[i,'BF_12M'] - 1
    # 절대 모멘텀 지표에 따라서 True / False 구분
    flag = True if((momentum_index > 0) and (momentum_index != np.inf) and (momentum_index != -np.inf)) else False
    if flag:
        signal ='buy'
    print('날짜: ', i, '모멘텀 인덱스:', momentum_index, 'flag :', flag,'signal:', signal)
    df.loc[i, 'trade'] = signal
df.value_counts('trade')


날짜:  1993-01-29 00:00:00 모멘텀 인덱스: nan flag : False signal: 
날짜:  1993-02-26 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-03-31 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-04-30 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-05-28 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-06-30 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-07-30 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-08-31 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-09-30 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-10-29 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-11-30 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1993-12-31 00:00:00 모멘텀 인덱스: inf flag : False signal: 
날짜:  1994-01-31 00:00:00 모멘텀 인덱스: 0.08709114326385659 flag : True signal: buy
날짜:  1994-02-28 00:00:00 모멘텀 인덱스: 0.1131289632117034 flag : True signal: buy
날짜:  1994-03-31 00:00:00 모멘텀 인덱스: 0.05699081889816049 flag : True signal: buy
날짜:  1994-04-29 00:00:00 모멘텀 인덱스: 0.03929402827

C:\Users\seopa\AppData\Local\Temp\ipykernel_9776\2014943034.py:10: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = month_last_df.loc[i, 'BF_1M'] / month_last_df.loc[i,'BF_12M'] - 1
C:\Users\seopa\AppData\Local\Temp\ipykernel_9776\2014943034.py:10: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = month_last_df.loc[i, 'BF_1M'] / month_last_df.loc[i,'BF_12M'] - 1


trade
       6401
buy     247
dtype: int64

In [12]:
# 수익률 계산
rtn = 1.0
df['return'] = 1
buy = 0
sell = 0
for i in df.index:
    # 구매한 날짜 체크 현재 trade = buy and 전 날 행의 trade =''
    if df.loc[i, 'trade'] =='buy' and df.shift(1).loc[i,'trade'] == '':
        buy = df.loc[i,'Adj Close']
        print('구매일:', i,'구매가격:', buy)
    elif df.loc[i,'trade'] == '' and df.shift(1).loc[i,'trade'] =='buy':
        sell = df.loc[i, 'Adj Close']
        rtn = (sell-buy) / buy + 1
        df.loc[i, 'return'] = rtn
        print('판매일:', i, '판매가격:', sell, '수익률:', rtn)

    if df.loc[i,'trade'] =='':
        buy = 0
        sell = 0
    

구매일: 1994-01-31 00:00:00 구매가격: 30.045223
판매일: 1994-02-01 00:00:00 판매가격: 29.889435 수익률: 0.9948148828850429
구매일: 1994-02-28 00:00:00 구매가격: 29.168985
판매일: 1994-03-01 00:00:00 판매가격: 29.052134 수익률: 0.995993998419897
구매일: 1994-03-31 00:00:00 구매가격: 27.946764
판매일: 1994-04-04 00:00:00 판매가격: 27.5159 수익률: 0.9845826872835794
구매일: 1994-04-29 00:00:00 구매가격: 28.260098
판매일: 1994-05-02 00:00:00 판매가격: 28.436377 수익률: 1.0062377349151443
구매일: 1994-05-31 00:00:00 구매가격: 28.710556
판매일: 1994-06-01 00:00:00 판매가격: 28.837847 수익률: 1.00443359578268
구매일: 1994-06-30 00:00:00 구매가격: 28.052668
판매일: 1994-07-01 00:00:00 판매가격: 28.111818 수익률: 1.0021085338478322
구매일: 1994-07-29 00:00:00 구매가격: 28.959522
판매일: 1994-08-01 00:00:00 판매가격: 29.097498 수익률: 1.0047644432805212
구매일: 1994-08-31 00:00:00 구매가격: 30.063478
판매일: 1994-09-01 00:00:00 판매가격: 29.964916 수익률: 0.9967215370091245
구매일: 1994-09-30 00:00:00 구매가격: 29.30423
판매일: 1994-10-03 00:00:00 판매가격: 29.234806 수익률: 0.9976309222252213
구매일: 1994-10-31 00:00:00 구매가격: 30.137241
판매일: 1994-1

In [13]:
acc_rtn = 1

for i in df.index:
    rtn = df.loc[i, 'return']
    acc_rtn *= rtn
    df.loc[i,'acc_rtn'] = acc_rtn

print('누적 수익률:', acc_rtn)

누적 수익률: 2.0162118510060383


# 절대 모멘텀 함수로 구현
1. 1번 함수
    - 매개 변수 2개 (데이터프레임, 기준이 되는 컬럼)
    - 데이터프레임에 결측치와 이상치 제거
    - Date, 기준이 되는 컬럼 두 컬럼만 두고 나머지 컬럼은 삭제
    - Date 컬럼의 데이터를 시계열데이터로 변경
    - 'STD_YM' 파생변수를 생성 해서 년-월 대입
    - 인덱스를 Date로 변경
    - 수정이 된 데이터프레임을 리턴
    - 1번 함수 테스트시 csv 파일 새로 불러와서 함수 호출
2. 2번 함수
    - 매개변수(df) 1개 (데이터프레임)
    - 새로운 데이터프레임 생성
    - df의 데이터에서 년-월별 마지막 데이터들을 새로운 데이터프레임 삽입
    - 새로운 데이터프레임에 파생변수 2개 생성
    - 'BF_1M' : 전월의 종가
    - 'BF_12M' : 전년의 종가
    - 새로운 데이터프레임을 리턴

In [14]:
# 1번 함수

def first(data, col):
    # 조건식을 이용하여 데이터형태를 같게 csv / yfinance
    #if type(data.index[0]) != int:
    if 'Date' not in data.columns: 
        data = data.reset_index()
        # data.reser_index(inplace=True)
    result = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)] # 결측치, 이상치 제거
    result = result.loc[:, ['Date', col]] # result[['Date', col]]
    result['Date'] = pd.to_datetime(result['Date'])
    result['STD-YM'] = result['Date'].dt.strftime('%Y-%m')
    result.set_index('Date', inplace=True)

    return result
    
    

In [15]:
df_1 = pd.read_csv('../csv/BND.csv')
df_1.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-04-10,75.070000,75.260002,75.000000,75.239998,51.523693,35000
1,2007-04-11,75.160004,75.290001,75.029999,75.040001,51.386742,87700
2,2007-04-12,75.059998,75.080002,74.959999,75.029999,51.379879,78100
3,2007-04-13,75.040001,75.070000,74.849998,74.910004,51.297688,18000
4,2007-04-16,74.989998,74.989998,74.940002,74.980003,51.345627,52700


In [16]:
test_df = first(df_1, 'Close')
test_df

C:\Users\seopa\AppData\Local\Temp\ipykernel_9776\2076908444.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  result = data[~data.isin([np.nan, np.inf, -np.inf]).any(1)] # 결측치, 이상치 제거


,Close,STD-YM
Date,,
2007-04-10,75.239998,2007-04
2007-04-11,75.040001,2007-04
2007-04-12,75.029999,2007-04
2007-04-13,74.910004,2007-04
2007-04-16,74.980003,2007-04
...,...,...
2019-06-18,82.589996,2019-06
2019-06-19,82.870003,2019-06
2019-06-20,83.000000,2019-06


In [17]:
# 2번 함수

def second(data):
    result = pd.DataFrame()
    col = data.columns[0]
    data = data[~(data.shift(-1)['STD-YM'] == data['STD-YM'])]
    result = pd.concat([result, data])
    result['BF_1M'] = result.shift(1)[col].fillna(0)
    result['BF_12M'] = result.shift(12)[col].fillna(0)
    return result

In [18]:
# def second(data):
#     col = data.columns[0]
#     result = data[~(data.shift(-1)['STD-YM'] == data['STD-YM'])]
#     result['BF_1M'] = result.shift(1)[col].fillna(0)
#     result['BF_12M'] = result.shift(12)[col].fillna(0)
#     return result
    
    

In [19]:
test_df_2 = second(test_df)
test_df_2

,Close,STD-YM,BF_1M,BF_12M
Date,,,,
2007-04-30,75.489998,2007-04,0.000000,0.000000
2007-05-31,74.769997,2007-05,75.489998,0.000000
2007-06-29,74.279999,2007-06,74.769997,0.000000
2007-07-31,74.730003,2007-07,74.279999,0.000000
2007-08-31,75.529999,2007-08,74.730003,0.000000
...,...,...,...,...
2019-02-28,79.820000,2019-02,80.089996,79.550003
2019-03-29,81.180000,2019-03,79.820000,79.930000
2019-04-30,80.959999,2019-04,81.180000,79.050003


3. 3번 함수 생성
    - 매개변수 2개 (데이터프레임(df1), 데이터프레임(df2))
    - df1에 'trade' 컬럼 생성 값은 ''
    - df1에 return 컬럼 생성 값은 1
    - df2의 값들을 이용하여 momentum index를 구하고 df1에 거래 내역 대입
    - df1의 거래내역을 이용하여 수익률 return 대입
    - return의 값들을 가지고 누적 수익률 acc_rtn 컬럼에 대입
    - 누적 수익률 print()
    - df1 을 return

In [20]:
# 3번 함수
def third(df1, df2):
    df1['trade'] = ''
    df1['return'] = 1
    
    for i in df2.index:
        signal = ''
        momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i,'BF_12M'] - 1
        flag = True if((momentum_index > 0) and (momentum_index != np.inf) and (momentum_index != -np.inf)) else False
        if flag:
            signal ='buy'
        df1.loc[i, 'trade'] = signal

    rtn = 1.0
    buy = 0
    sell = 0
    col = df1.columns[0]

    for i in df1.index:
        # 구매한 날짜 체크 현재 trade = buy and 전 날 행의 trade =''
        if df1.loc[i, 'trade'] =='buy' and df1.shift(1).loc[i,'trade'] == '':
            buy = df1.loc[i, col]
            print('구매일:', i,'구매가격:', buy)
        elif df1.loc[i,'trade'] == '' and df1.shift(1).loc[i,'trade'] =='buy':
            sell = df1.loc[i, col]
            rtn = (sell-buy) / buy + 1
            df1.loc[i, 'return'] = rtn
            print('판매일:', i, '판매가격:', sell, '수익률:', rtn)

        if df1.loc[i,'trade'] =='':
            buy = 0
            sell = 0

    acc_rtn = 1

    for i in df1.index:
        rtn = df1.loc[i, 'return']
        acc_rtn *= rtn
        df1.loc[i,'acc_rtn'] = acc_rtn

    print('누적 수익률:', acc_rtn)

    return df1

In [21]:
third(test_df,test_df_2)

C:\Users\seopa\AppData\Local\Temp\ipykernel_9776\1679978655.py:8: RuntimeWarning: invalid value encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i,'BF_12M'] - 1
C:\Users\seopa\AppData\Local\Temp\ipykernel_9776\1679978655.py:8: RuntimeWarning: divide by zero encountered in scalar divide
  momentum_index = df2.loc[i, 'BF_1M'] / df2.loc[i,'BF_12M'] - 1


구매일: 2008-04-30 00:00:00 구매가격: 77.57
판매일: 2008-05-01 00:00:00 판매가격: 77.279999 수익률: 0.9962614283872633
구매일: 2008-05-30 00:00:00 구매가격: 76.5
판매일: 2008-06-02 00:00:00 판매가격: 76.589996 수익률: 1.0011764183006535
구매일: 2008-06-30 00:00:00 구매가격: 76.25
판매일: 2008-07-01 00:00:00 판매가격: 75.900002 수익률: 0.995409862295082
구매일: 2008-07-31 00:00:00 구매가격: 75.970001
판매일: 2008-08-01 00:00:00 판매가격: 75.800003 수익률: 0.9977623009377083
구매일: 2008-08-29 00:00:00 구매가격: 76.300003
판매일: 2008-09-02 00:00:00 판매가격: 76.260002 수익률: 0.9994757405186471
구매일: 2008-09-30 00:00:00 구매가격: 75.620003
판매일: 2008-10-01 00:00:00 판매가격: 74.949997 수익률: 0.9911398310841114
구매일: 2009-01-30 00:00:00 구매가격: 77.330002
판매일: 2009-02-02 00:00:00 판매가격: 77.300003 수익률: 0.9996120651852565
구매일: 2009-05-29 00:00:00 구매가격: 77.480003
판매일: 2009-06-01 00:00:00 판매가격: 76.690002 수익률: 0.9898038078289699
구매일: 2009-06-30 00:00:00 구매가격: 77.690002
판매일: 2009-07-01 00:00:00 판매가격: 77.379997 수익률: 0.9960097182131621
구매일: 2009-07-31 00:00:00 구매가격: 78.440002
판매일: 2009-08-03 00:

,Close,STD-YM,trade,return,acc_rtn
Date,,,,,
2007-04-10,75.239998,2007-04,,1.0,1.000000
2007-04-11,75.040001,2007-04,,1.0,1.000000
2007-04-12,75.029999,2007-04,,1.0,1.000000
2007-04-13,74.910004,2007-04,,1.0,1.000000
2007-04-16,74.980003,2007-04,,1.0,1.000000
...,...,...,...,...,...
2019-06-18,82.589996,2019-06,,1.0,0.826513
2019-06-19,82.870003,2019-06,,1.0,0.826513
2019-06-20,83.000000,2019-06,,1.0,0.826513


In [124]:
import momentum as mm
import imp
imp.reload(mm)

<module 'momentum' from 'c:\\Users\\ssjjk\\Desktop\\6기\\221222\\momentum\\__init__.py'>

In [125]:
df = pd.read_csv('../csv/AMZN.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000
1,1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000
2,1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800
3,1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200
4,1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200


In [126]:
mmc = mm.Momentum(df, 'Adj Close')

In [127]:
mmc.testing()

c:\Users\ssjjk\Desktop\6기\221222\momentum\__init__.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  self.df = self.df[~self.df.isin([np.nan, np.inf, -np.inf]).any(1)] # 결측치, 이상치 제거
c:\Users\ssjjk\Desktop\6기\221222\momentum\__init__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df2['BF_1M'] = self.df2.shift(1)[self.col].fillna(0)
c:\Users\ssjjk\Desktop\6기\221222\momentum\__init__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

구매일: 1998-05-29 00:00:00 구매가격: 7.34375
판매일: 1998-06-01 00:00:00 판매가격: 7.135417 수익률: 0.9716312510638299
구매일: 1998-06-30 00:00:00 구매가격: 16.625
판매일: 1998-07-01 00:00:00 판매가격: 19.020834 수익률: 1.1441103157894736
구매일: 1998-07-31 00:00:00 구매가격: 18.479166
판매일: 1998-08-03 00:00:00 판매가격: 18.041666 수익률: 0.976324689112052
구매일: 1998-08-31 00:00:00 구매가격: 13.958333
판매일: 1998-09-01 00:00:00 판매가격: 13.325517 수익률: 0.9546639272755565
구매일: 1998-09-30 00:00:00 구매가격: 18.604166
판매일: 1998-10-01 00:00:00 판매가격: 17.104166 수익률: 0.9193728974467331
구매일: 1998-10-30 00:00:00 구매가격: 21.072916
판매일: 1998-11-02 00:00:00 판매가격: 21.833334 수익률: 1.0360850866581541
구매일: 1998-11-30 00:00:00 구매가격: 32.0
판매일: 1998-12-01 00:00:00 판매가격: 34.916668 수익률: 1.091145875
구매일: 1998-12-31 00:00:00 구매가격: 53.541668
판매일: 1999-01-04 00:00:00 판매가격: 59.15625 수익률: 1.1048637857154544
구매일: 1999-01-29 00:00:00 구매가격: 58.46875
판매일: 1999-02-01 00:00:00 판매가격: 57.9375 수익률: 0.9909139497594869
구매일: 1999-02-26 00:00:00 구매가격: 64.0625
판매일: 1999-03-01 00:00:00 판매가격:

,Adj Close,STD-YM,trade,return,acc_rtn
Date,,,,,
1997-05-15,1.958333,1997-05,,1.0,1.00000
1997-05-16,1.729167,1997-05,,1.0,1.00000
1997-05-19,1.708333,1997-05,,1.0,1.00000
1997-05-20,1.635417,1997-05,,1.0,1.00000
1997-05-21,1.427083,1997-05,,1.0,1.00000
...,...,...,...,...,...
2019-06-18,1901.369995,2019-06,,1.0,1.03362
2019-06-19,1908.790039,2019-06,,1.0,1.03362
2019-06-20,1918.189941,2019-06,,1.0,1.03362


In [22]:
import yfinance as yf

In [23]:
yf_data = yf.download('005930.KS',start='2019-04-11')

[*********************100%***********************]  1 of 1 completed


In [24]:
mmc = mm.Momentum(yf_data, 'Adj Close')

NameError: name 'mm' is not defined

In [132]:
mmc.testing()

c:\Users\ssjjk\Desktop\6기\221222\momentum\__init__.py:14: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  self.df = self.df[~self.df.isin([np.nan, np.inf, -np.inf]).any(1)] # 결측치, 이상치 제거
c:\Users\ssjjk\Desktop\6기\221222\momentum\__init__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df2['BF_1M'] = self.df2.shift(1)[self.col].fillna(0)
c:\Users\ssjjk\Desktop\6기\221222\momentum\__init__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

구매일: 2020-04-29 00:00:00 구매가격: 46440.5078125
판매일: 2020-05-04 00:00:00 판매가격: 45047.296875 수익률: 0.9700000925242898
구매일: 2020-05-29 00:00:00 구매가격: 47090.68359375
판매일: 2020-06-01 00:00:00 판매가격: 47555.08984375 수익률: 1.0098619560082505
구매일: 2020-06-30 00:00:00 구매가격: 49369.0703125
판매일: 2020-07-01 00:00:00 판매가격: 49182.0703125 수익률: 0.996212203332647
구매일: 2020-07-31 00:00:00 구매가격: 54137.671875
판매일: 2020-08-03 00:00:00 판매가격: 53109.1484375 수익률: 0.981001705432129
구매일: 2020-08-31 00:00:00 구매가격: 50491.1015625
판매일: 2020-09-01 00:00:00 판매가격: 50678.09765625 수익률: 1.0037035455350192
구매일: 2020-09-29 00:00:00 구매가격: 54752.9375
판매일: 2020-10-05 00:00:00 판매가격: 55223.3203125 수익률: 1.008591005962009
구매일: 2020-10-30 00:00:00 구매가격: 53247.70703125
판매일: 2020-11-02 00:00:00 판매가격: 54000.32421875 수익률: 1.0141342647310296
구매일: 2020-11-30 00:00:00 구매가격: 62749.50390625
판매일: 2020-12-01 00:00:00 판매가격: 63784.34765625 수익률: 1.0164916642456026
구매일: 2020-12-30 00:00:00 구매가격: 78120.3046875
판매일: 2021-01-04 00:00:00 판매가격: 80049.2265625

,Adj Close,STD-YM,trade,return,acc_rtn
Date,,,,,
2019-04-11,41739.449219,2019-04,,1.0,1.000000
2019-04-12,42280.929688,2019-04,,1.0,1.000000
2019-04-15,42461.425781,2019-04,,1.0,1.000000
2019-04-16,42641.914062,2019-04,,1.0,1.000000
2019-04-17,42461.425781,2019-04,,1.0,1.000000
...,...,...,...,...,...
2022-12-16,59500.000000,2022-12,,1.0,1.112359
2022-12-19,59500.000000,2022-12,,1.0,1.112359
2022-12-20,58600.000000,2022-12,,1.0,1.112359
